In [ ]:
import json
import pandas as pd
import os
import shutil
from glob import glob
import zipfile
import numpy as np
from tqdm import tqdm
from shapely import wkb
from shapely.geometry import shape
import geopandas as gpd
from shapely.geometry import Point

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
QV_THRESHOLD = 20 

In [ ]:
# NCBI 마커 gene list
marker_genes = {
    "epithelial": [
        # Epithelial 기본 마커
        "EPCAM", "KRT8", "KRT18", "KRT19", "KRT7",
        # Luminal 마커
        "ERBB2", "ESR1", "PGR", "GATA3", "FOXA1",
        # Proliferation
        "MKI67", "PCNA",
        # Junction
        "CDH1", "CLDN4", "CLDN3", "CLDN7",
        # 추가 상피 마커
        "KRT5", "KRT14", "TP63"
    ],

    "Basal/Myoepithelial": [
        # Basal specific
        "KRT5", "KRT14", "KRT17", "TP63", "KRT15",
        # Myoepithelial specific
        "ACTA2", "MYH11", "MYLK", "TAGLN", "CNN1", "MYL9",
        "OXTR", "CD10", "SMA"
    ],

    "Smooth muscle": [
        # Smooth muscle specific (vascular)
        "ACTA2", "MYH11", "MYLK", "TAGLN", "CNN1", "MYL9",
        "ACTG2", "DES", "SMTN", "LMOD1"
    ],

    "Fibroblast": [
        # CAF (Cancer-Associated Fibroblast) 마커
        "PDGFRA", "PDGFRB", "FAP", "S100A4", "ACTA2",
        # ECM 관련
        "COL1A1", "COL1A2", "COL3A1", "COL5A1", "COL6A1",
        "FN1", "VIM", "DCN", "LUM",
        # Fibroblast specific
        "DPT", "SFRP1", "FBLN1", "SFRP4", "POSTN",
        "THY1", "CDH2", "PDPN"
    ],

    "Endothelial": [
        # Pan-endothelial
        "PECAM1", "CD34", "CDH5", "VWF", "VCAM1",
        # Vascular
        "KDR", "FLT1", "CD93", "EGFL7",
        # Capillary/Lymphatic
        "CLEC14A", "MMRN2", "ESM1", "PLVAP",
        "ACKR1", "SELE", "TEK"
    ],

    "Lymphocyte": [
        # T cell 범용
        "CD3D", "CD3E", "CD3G", "TRAC", "TRBC1", "TRBC2",
        # T cell subtype
        "CD4", "CD8A", "CD8B",
        # Activation/Function
        "GZMA", "GZMB", "GZMK", "NKG7", "CCL5", "PRF1",
        "IFNG", "TNF", "IL2RG", "CXCR3",
        # Naive/Memory
        "TCF7", "LEF1", "CCR7", "SELL", "IL7R",
        # B cell
        "CD19", "CD79A", "CD79B", "MS4A1", "CD20",
        "PAX5", "BLK", "BANK1",
        # NK cell
        "NCAM1", "KLRD1", "KLRB1", "GNLY"
    ],

    "Plasma cell": [
        "MZB1", "SDC1", "CD38", "PRDM1", "XBP1",
        "TNFRSF17", "SLAMF7", "JCHAIN", "IRF4",
        "IGHA1", "IGHA2", "IGHG1", "IGHG2", "IGHG3", "IGHG4",
        "SSR4", "DERL3"
    ],

    "Macrophage/Histiocyte": [
        # Pan-macrophage
        "CD68", "CD163", "CSF1R", "AIF1", "ITGAM",
        # M2 macrophage
        "MRC1", "MSR1", "CD14", "FCGR3A",
        # Activation
        "C1QA", "C1QB", "C1QC", "APOE", "SPP1",
        # Tissue resident
        "CX3CR1", "LST1", "TYROBP", "FCER1G",
        "CD74", "HLA-DRA", "HLA-DRB1"
    ],

    "Neutrophil": [
        "S100A8", "S100A9", "S100A12", "LYZ",
        "CEACAM8", "MPO", "ELANE", "CTSG",
        "CSF3R", "FCGR3B", "CXCR2", "CXCR1",
        "MMP9", "CAMP"
    ],

    "Adipocyte": [
        "ADIPOQ", "LEP", "LPL", "PPARG", "CEBPA",
        "FABP4", "PLIN1", "PLIN2", "CIDEC",
        "FASN", "SCD", "RETN", "RBP4"
    ],

    "Other/Unknown": []
}
class_list = {
    0: "epithelial",
    1: "Basal/Myoepithelial",
    2: "Smooth muscle",
    3: "Fibroblast",
    4: "Endothelial",
    5: "Lymphocyte",                # T + B 통합
    6: "Plasma cell",
    7: "Macrophage/Histiocyte",     # 통합
    8: "Neutrophil",
    9: "Adipocyte",
    10: "Other/Unknown"
}
class_colors_hex = {
    "epithelial": "#FF0000",        # 빨강
    "Basal/Myoepithelial": "#FFA500",     # 주황
    "Smooth muscle": "#8B4513",           # 갈색
    "Fibroblast": "#00FF00",              # 초록
    "Endothelial": "#0000FF",             # 파랑
    "Lymphocyte": "#FFFF00",              # 노랑 (T/B lymphocyte 통합)
    "Plasma cell": "#9400D3",             # 보라
    "Macrophage/Histiocyte": "#00FFFF",   # 시안(청록)
    "Neutrophil": "#1E90FF",              # DodgerBlue (밝은 파랑)
    "Adipocyte": "#FFC0CB",               # 핑크
    "Other/Unknown": "#808080"            # 회색
}

class_colors = {
    "epithelial": [255, 0, 0],            # 빨강 - 종양 상피
    "Basal/Myoepithelial": [255, 165, 0],       # 주황
    "Smooth muscle": [139, 69, 19],             # 갈색
    "Fibroblast": [0, 255, 0],                  # 초록
    "Endothelial": [0, 0, 255],                 # 파랑
    "Lymphocyte": [255, 255, 0],                # 노랑 (T/B 통합)
    "Plasma cell": [148, 0, 211],               # 보라
    "Macrophage/Histiocyte": [0, 255, 255],     # 시안 (청록)
    "Neutrophil": [30, 144, 255],               # 도저블루 (밝은 파랑)
    "Adipocyte": [255, 192, 203],               # 핑크
    "Other/Unknown": [128, 128, 128]            # 회색
}

In [ ]:
# TENX 마커 gene list
marker_genes = {
    "epithelial": [
        # Epithelial 기본 마커
        "EPCAM", "KRT8", "KRT18", "KRT19", "KRT7",
        "TACSTD2", "CLDN4",
        # Luminal 마커
        "ERBB2", "ESR1", "PGR", "GATA3",
        # Proliferation
        "MKI67", "PCNA",
        # Junction
        "CDH1",
        # Keratin
        "KRT5", "KRT14", "KRT15", "KRT16", "KRT17", "KRT20", "KRT23", "KRT80",
        # 추가
        "MSLN", "WFDC2"
    ],

    "Basal/Myoepithelial": [
        # Basal specific
        "KRT5", "KRT14", "KRT15", "KRT17", "TP63",
        # Myoepithelial specific
        "ACTA2", "TAGLN", "MYH11", "CNN1",
        # 추가
        "PDGFRB", "RGS5", "COL17A1", "ITGA6", "LAMC2"
    ],

    "Fibroblast": [
        # ECM 관련
        "COL1A1", "COL1A2", "COL3A1", "COL5A2", 
        "COL6A1", "COL6A2", "COL11A1",
        "DCN", "LUM", "BGN", "SPARC", "FN1",
        # Fibroblast specific
        "PDGFRA", "FAP", "CXCL12", "DPT",
        # CAF 마커
        "MMP2", "MMP14",
        # 추가
        "C7", "C1R", "C1S", "FBLN1", "POSTN", "THY1"
    ],

    "Endothelial": [
        # Pan-endothelial
        "PECAM1", "VWF", "CDH5", "CD34", "CLDN5",
        # Vascular
        "KDR", "FLT1", "ENG", "EGFL7",
        # Capillary/Lymphatic
        "PLVAP", "ACKR1", "PROX1", "LYVE1",
        # 추가
        "RAMP2", "SELE", "ECSCR"
    ],

    "Lymphocyte": [
        # T cell 범용
        "PTPRC", "CD3D", "CD3E", "CD3G", 
        "TRAC", "TRBC1", "TRBC2",
        # T cell markers
        "CD4", "CD8A", "CD8B",
        "IL7R", "CCR7", "LTB",
        # Cytotoxic/NK
        "NKG7", "GNLY", "PRF1", "GZMK", "GZMB", "GZMH", "KLRD1",
        # B cell
        "MS4A1", "CD79A", "CD79B", "CD19", "CD74",
        # 추가
        "CD2", "CD5", "CD7", "CD27", "CD28", "CD52", "CD69"
    ],

    "Plasma cell": [
        "MZB1", "XBP1", "JCHAIN",
        "SDC1", "TNFRSF17",
        # Immunoglobulin
        "IGKC", "IGLC3",
        "IGHG1", "IGHG3", "IGHG4", "IGHM",
        # 추가
        "DERL3", "SEC11C", "SSR4"
    ],

    "Macrophage/Histiocyte": [
        # Pan-macrophage
        "LYZ", "CD68", "CSF1R", "SPI1", "AIF1",
        # M2/Tissue resident
        "CD163", "MRC1", "MS4A7", "FCGR3A",
        # Complement
        "C1QA", "C1QB", "C1QC",
        # 추가
        "TYROBP", "FCER1G", "CTSS", "LST1",
        "MARCO", "APOE", "TREM2", "IL1B", "CD14"
    ],

    "Neutrophil": [
        "S100A8", "S100A9", "S100A12",
        "CXCR2", "CXCR1",
        "CTSG", "LTF", "LCN2",
        "MNDA", "CSF3R"
    ],

    "Adipocyte": [
        "ADIPOQ", "FABP4", "LPL", "PPARG", "RBP4",
        "LEP", "PLIN4"
    ],

    "Other/Unknown": []
}
class_list = {
    0: "epithelial",
    1: "Basal/Myoepithelial",
    2: "Smooth muscle",
    3: "Fibroblast",
    4: "Endothelial",
    5: "Lymphocyte",                # T + B 통합
    6: "Plasma cell",
    7: "Macrophage/Histiocyte",     # 통합
    8: "Neutrophil",
    9: "Adipocyte",
    10: "Other/Unknown"
}
class_colors_hex = {
    "epithelial": "#FF0000",        # 빨강
    "Basal/Myoepithelial": "#FFA500",     # 주황
    "Smooth muscle": "#8B4513",           # 갈색
    "Fibroblast": "#00FF00",              # 초록
    "Endothelial": "#0000FF",             # 파랑
    "Lymphocyte": "#FFFF00",              # 노랑 (T/B lymphocyte 통합)
    "Plasma cell": "#9400D3",             # 보라
    "Macrophage/Histiocyte": "#00FFFF",   # 시안(청록)
    "Neutrophil": "#1E90FF",              # DodgerBlue (밝은 파랑)
    "Adipocyte": "#FFC0CB",               # 핑크
    "Other/Unknown": "#808080"            # 회색
}

class_colors = {
    "epithelial": [255, 0, 0],            # 빨강 - 종양 상피
    "Basal/Myoepithelial": [255, 165, 0],       # 주황
    "Smooth muscle": [139, 69, 19],             # 갈색
    "Fibroblast": [0, 255, 0],                  # 초록
    "Endothelial": [0, 0, 255],                 # 파랑
    "Lymphocyte": [255, 255, 0],                # 노랑 (T/B 통합)
    "Plasma cell": [148, 0, 211],               # 보라
    "Macrophage/Histiocyte": [0, 255, 255],     # 시안 (청록)
    "Neutrophil": [30, 144, 255],               # 도저블루 (밝은 파랑)
    "Adipocyte": [255, 192, 203],               # 핑크
    "Other/Unknown": [128, 128, 128]            # 회색
}

In [ ]:
def classify_cell_by_genes(gene_list, marker_dict, min_markers=1, min_score_ratio=0.5):
    """
    여러 유전자를 기반으로 cell type scoring
    
    Args:
        gene_list: 셀 내 검출된 유전자 리스트
        marker_dict: cell type별 marker gene 딕셔너리
        min_markers: cell type으로 분류하기 위한 최소 marker 수
        min_score_ratio: 2등과의 점수 비율 (0.5 = 2등이 1등의 50% 이상이면 애매함)
    """
    scores = {cell_type: 0 for cell_type in marker_dict.keys()}
    
    # 각 gene이 어느 cell type의 marker인지 카운트
    for gene in gene_list:
        for cell_type, markers in marker_dict.items():
            if gene in markers:
                scores[cell_type] += 1
    
    # 점수 정렬
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    max_score = sorted_scores[0][1]
    second_score = sorted_scores[1][1] if len(sorted_scores) > 1 else 0
    
    # 최소 marker 수 미만이면 Unknown
    if max_score < min_markers:
        return 'Other/Unknown', scores
    
    # 2등과 점수 차이가 작으면 애매한 경우
    if second_score > 0 and second_score / max_score >= min_score_ratio:
        # 동점에 가까운 경우 우선순위 적용
        tied_types = [ct for ct, score in scores.items() 
                     if score >= max_score * min_score_ratio]
        
        priority = [
            "epithelial", "Cycling", "Lymphocyte", "Plasma cell",
            "Macrophage/Histiocyte", "Neutrophil", "CAF_like",
            "Fibroblast", "Pericyte/SMC", "Endothelial",
            "Basal/Myoepithelial", "Smooth muscle", "Adipocyte"
        ]
        
        for cell_type in priority:
            if cell_type in tied_types:
                return cell_type, scores
    
    # 명확한 1등
    best_type = max(scores, key=scores.get)
    return best_type, scores

xenium_annotation_list = glob('../../data/spatialTranscriptome/cellvit_seg/*_cellvit_seg.parquet')
save_path = '../../data/spatialTranscriptome/preprocessed_xenium/'
xenium_transcripts_list = [f.replace("cellvit_seg/", "transcripts/") for f in xenium_annotation_list]
xenium_transcripts_list = [f.replace("_cellvit_seg.parquet", "_transcripts.parquet") for f in xenium_transcripts_list]

for i in range(len(xenium_transcripts_list)):
    if os.path.exists(save_path + 'labels/' + os.path.basename(xenium_annotation_list[i]).replace('_cellvit_seg.parquet', '.csv')):
        print(f"Skipping (already processed): {save_path + 'labels/' + os.path.basename(xenium_annotation_list[i]).replace('_cellvit_seg.parquet', '.csv')}")
        continue
    
    xenium_transcript_path = xenium_transcripts_list[i]
    df_transcript = pd.read_parquet(xenium_transcript_path)
    df_filtered = df_transcript[df_transcript['qv'] > QV_THRESHOLD].copy()
    
    # feature_name 처리
    if type(df_filtered['feature_name'].iloc[0]) == bytes:
        df_filtered['feature_name'] = df_filtered['feature_name'].str.decode('utf-8')
        df_filtered = df_filtered[~df_filtered['feature_name'].str.contains('BLANK|NegControl|antisense', case=False, na=False)]
    elif type(df_filtered['feature_name'].iloc[0]) == str:
        df_filtered = df_filtered[~df_filtered['feature_name'].str.contains('BLANK|NegControl|antisense', case=False, na=False)]
    else:
        print(f"Skipping: {xenium_transcript_path}")
        continue
    
    # Segmentation 데이터 로드
    xenium_annotation_path = xenium_annotation_list[i]
    df_seg = pd.read_parquet(xenium_annotation_path)
    
    # GeoDataFrame 변환
    transcript_gdf = gpd.GeoDataFrame(
        df_filtered,
        geometry=gpd.points_from_xy(df_filtered['he_x'], df_filtered['he_y'])
    )
    
    polygons = [wkb.loads(geom) for geom in df_seg['geometry']]
    seg_gdf = gpd.GeoDataFrame(df_seg, geometry=polygons)
    
    # Spatial join 수행
    joined = gpd.sjoin(transcript_gdf, seg_gdf, how='inner', predicate='within')
    
    # 🚀 최적화: groupby로 한 번에 처리
    annotations = []
    
    if len(joined) > 0:
        # index_right로 그룹화하여 각 셀의 유전자 리스트 한 번에 생성
        grouped = joined.groupby('index_right')['feature_name'].apply(list)
        
        for idx in tqdm(range(len(seg_gdf))):
            if idx in grouped.index:
                genes_in_cell = grouped[idx]
                cell_type, score = classify_cell_by_genes(genes_in_cell, marker_genes)
                
                bounds = seg_gdf.iloc[idx].geometry.bounds
                annotations.append({
                    'x1': int(bounds[0]),
                    'y1': int(bounds[1]),
                    'x2': int(bounds[2]),
                    'y2': int(bounds[3]),
                    'class_name': cell_type,
                    'gene_count': len(genes_in_cell)
                })
    
    # 저장
    if annotations:
        df = pd.DataFrame(annotations)
        
        # 통계 출력
        print(f"\n=== {os.path.basename(xenium_annotation_path)} ===")
        print(f"Total cells: {len(df)}")
        print(f"\nCell type distribution:")
        type_counts = df['class_name'].value_counts()
        for cell_type, count in type_counts.items():
            percentage = (count / len(df)) * 100
            print(f"  {cell_type}: {count} ({percentage:.1f}%)")
        
        # Unknown 비율이 높으면 경고
        unknown_ratio = type_counts.get('Other/Unknown', 0) / len(df)
        if unknown_ratio > 0.5:
            print(f"\n⚠️ WARNING: {unknown_ratio*100:.1f}% cells are Unknown!")
            print("Consider:")
            print("  1. Lowering min_markers threshold")
            print("  2. Adding more marker genes")
            print("  3. Checking transcript quality")
        
        # 저장
        create_dir(save_path + 'labels/')
        output_file = save_path + 'labels/' + os.path.basename(xenium_annotation_path).replace('_cellvit_seg.parquet', '.csv')
        df.to_csv(output_file, index=False)

print("All samples processed!")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import openslide as ops

reduction_factor=20
slide=ops.OpenSlide('../../data/spatialTranscriptome/preprocessed_xenium/wsis/TENX106.tif')
thumbnail = slide.get_thumbnail((slide.level_dimensions[0][0] // reduction_factor, slide.level_dimensions[0][1] // reduction_factor))
mask=np.ones_like(np.array(thumbnail)) * 0
labels_df=pd.read_csv('../../data/spatialTranscriptome/preprocessed_xenium/labels/TENX106.csv')

fig, ax = plt.subplots(figsize=(22, 20))

for idx, row in labels_df.iterrows():
    x=row['x1']/reduction_factor + row['x2']/reduction_factor
    x=x//2
    y=row['y1']/reduction_factor + row['y2']/reduction_factor
    y=y//2
    
    mask[int(y):int(y)+2, int(x):int(x)+2]=np.array([class_colors[row['class_name']]])/255.

ax.imshow(mask*0.5 + np.array(thumbnail)/255.*0.5)
ax.axis('off')
ax.set_title('Cell Type Annotation', fontsize=16, fontweight='bold')

# 클래스별 개수 계산
class_counts = labels_df['class_name'].value_counts()

# 범례 추가 (클래스 개수 포함)
legend_patches = []
for class_name, hex_color in class_colors_hex.items():
    count = class_counts.get(class_name, 0)
    label = f"{class_name}: {count}"
    patch = mpatches.Patch(color=hex_color, label=label)
    legend_patches.append(patch)

ax.legend(handles=legend_patches, 
         loc='upper right', 
         fontsize=15,
         framealpha=0.95,
         bbox_to_anchor=(1.18, 1.0),
         title='Cell Type (Count)',
         title_fontsize=12)

plt.tight_layout()
plt.show()

# 전체 통계 출력
print("=== Cell Type Statistics ===")
print(f"Total cells: {len(labels_df)}")
print("\nCell type distribution:")
print(class_counts.sort_index())

In [ ]:
df_filtered[df_filtered['cell_id']==b'aaaeppaj-1']

In [ ]:
grouped_transcripts